# BEM114 Homework 3 - Value of Intangibles
**Names:** Andrew Zabelo, Daniel Wen, Kyle McCandless  
**UIDs:** , 2159859, 2157818

In [3]:
# Imports
import pandas as pd
import numpy as np
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

import matplotlib.pyplot as plt

## Problem 1

In [21]:
# load and clean up data
df = pd.read_excel("bcwlist.xlsx", sheet_name="bcwlist", engine="openpyxl")
df.columns = df.iloc[1]
df = df.drop([0,1])
df = df.iloc[:, 5:]
df = df.dropna(subset=['permno'])
df

1,rank,company,permno,year
2,1,AT&T Bell Laboratories,66093,1984
3,2,Trammell Crow Company,85629,1984
4,3,Delta Airlines,26112,1984
5,4,Federal Express,60628,1984
6,5,Goldman Sachs,86868,1984
...,...,...,...,...
2488,87,AbbVie,13721,2020
2489,88,Encompass Home Health & Hospice,10693,2020
2495,94,Goldman Sachs,86868,2020
2500,99,Delta Airlines,91926,2020


In [29]:
# all available firms for each year
avail_years = [1984, 1993]
avail_years = avail_years + [i for i in range(1998, 2021)]
avail_years = np.array(avail_years)
firms = dict()
for i in range(1984, 2021):
    year = i
    if year not in avail_years:
        year = np.max(avail_years[avail_years < year])
    filtered_df = df[df['year'] == year]
    firms[i] = set(filtered_df['permno'])

firms

{1984: {10604,
  11607,
  11703,
  11754,
  11850,
  12060,
  12458,
  12490,
  13119,
  14593,
  15560,
  15720,
  17144,
  18163,
  18403,
  19350,
  19692,
  22111,
  22592,
  23077,
  24459,
  24766,
  26083,
  26112,
  26438,
  26542,
  27633,
  27828,
  27887,
  28126,
  30681,
  32054,
  34817,
  37875,
  38914,
  39917,
  40061,
  40483,
  41080,
  43916,
  44062,
  45866,
  47079,
  47837,
  48071,
  48960,
  49154,
  49306,
  49411,
  52090,
  52564,
  54114,
  54391,
  55976,
  57681,
  57817,
  57947,
  59061,
  59184,
  59328,
  59425,
  60572,
  60628,
  60839,
  60871,
  61241,
  61399,
  61807,
  62069,
  64013,
  64346,
  65569,
  66093,
  67029,
  74617,
  75288,
  76085,
  76766,
  79374,
  80814,
  82486,
  82736,
  83601,
  85629,
  85796,
  86868,
  89431},
 1985: {10604,
  11607,
  11703,
  11754,
  11850,
  12060,
  12458,
  12490,
  13119,
  14593,
  15560,
  15720,
  17144,
  18163,
  18403,
  19350,
  19692,
  22111,
  22592,
  23077,
  24459,
  24766,
  2608

In [70]:
# add date when should be considered
df = pd.read_csv('crsp_1926_2020.csv')

# Convert prices and returns to numeric and drop NaNs
df['PRC'] = pd.to_numeric(df['PRC'], errors='coerce')
df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
df = df.dropna(subset=['PRC', 'RET'])

# removes first appearance of each permno
df = df[df.groupby('PERMNO').cumcount() != 0]
df['MV'] = abs(df['PRC'] * df['SHROUT'])


# [From HW2] Filter SHRCD and EXCHCD, remove negative prices
df = df[df['SHRCD'].isin([10, 11])]
df = df[df['EXCHCD'].isin([1, 2, 3])]
df = df[df['PRC'] >= 0]
df = df.drop(['SHRCD', 'EXCHCD', 'PRC', 'SHROUT'], axis=1)

df['date'] =  df['date'].str[:-3]

df = df.sort_values(by='date')


In [71]:
new_df = pd.DataFrame(columns=df.columns)
year = 1984
month = 1
while year < 2021:
    if year % 5 == 0:
        print(year)
    while month <= 12:
        date = f'{year}-{month:02d}'
        if month == 1 and year != 1984:
            firm = firms[year - 1]
        else:
            firm = firms[year]

        filtered_df = df[df['date'] == date]
        filtered_df = filtered_df[filtered_df['PERMNO'].isin(firm)]

        new_df = new_df.append(filtered_df, ignore_index=True)
        
        # iterate to next month
        month += 1
    month = 1
    year += 1

new_df

/var/folders/wc/bjtjbszx27s7y1w8qfl73z7w0000gn/T/ipykernel_8272/3936944455.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(filtered_df, ignore_index=True)


1985
1990
1995
2000
2005
2010
2015
2020


,PERMNO,date,RET,MV
0,74617,1984-01,-0.103203,1.257039e+06
1,15560,1984-01,-0.190202,3.663502e+06
2,65569,1984-01,0.024026,3.847984e+05
3,22592,1984-01,-0.060606,9.085712e+06
4,52090,1984-01,0.033058,3.412812e+05
...,...,...,...,...
22977,17342,2020-12,0.111167,7.151837e+06
22978,85913,2020-12,0.039804,4.279669e+07
22979,16304,2020-12,0.163563,6.058009e+06
22980,77660,2020-12,0.159676,5.821729e+06


In [ ]:
def calc_weights(group):
    # Calc equal weights
    group['weights_eq'] = 1 / float(group['decile'].count())
    # Calc total market equity of group
    group['TME'] = group['ME'].sum()
    # Calc value weights
    group['weights_val'] = group['ME'] / group['TME']
    return group

In [122]:
import warnings

ret_df = pd.DataFrame(columns=df.columns)

year = 1984
month = 2
date = f'{year}-{month:02d}'
prev_weights = None

while year < 2021:
    if year % 5 == 0:
        print(year)
    while month <= 12:
        date = f'{year}-{month:02d}'
        prev_month = month - 1
        prev_year = year
        if prev_month == 0:
            prev_month = 12
            prev_year -= 1 
        prev_date = f'{prev_year}-{prev_month:02d}'

        prev_filtered_df = new_df[new_df['date'] == prev_date]
        filtered_df = new_df[new_df['date'] == date].sort_values(by='PERMNO').reset_index()

        prev_firms = set(prev_filtered_df['PERMNO'])
        total_firms = len(prev_firms)
        firms = set(filtered_df['PERMNO'])

        diff = (prev_firms - firms).union(firms - prev_firms)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
        
            filtered_df['weights_eq'] = 1 / float(len(filtered_df))

            diff = (prev_firms - firms).union(firms - prev_firms)
            if len(diff) > 0 or month == 2 or (year == 1984 and month == 2):
                print("HERE1")
                filtered_df['TMV'] = filtered_df['MV'].sum()
                filtered_df['weights_val'] = filtered_df['MV'] / filtered_df['TMV']
                prev_weights = filtered_df['weights_val']
            else:
                print("HERE")
                filtered_df['TMV'] = filtered_df['MV'].sum()
                print(prev_weights)
                filtered_df['weights_val'] = prev_weights
                print(filtered_df)


        ret_df = pd.concat([ret_df, filtered_df])

        # iterate to next month
        month += 1
    month = 1
    year += 1

ret_df.to_csv('ret_df.csv', index=False)


HERE1
HERE
0     0.040119
1     0.004965
2     0.040396
3     0.040148
4     0.116281
        ...   
63    0.001850
64    0.001300
65    0.004843
66    0.000150
67    0.001437
Name: weights_val, Length: 68, dtype: float64
    index PERMNO     date       RET            MV  weight_eq  weight_val  \
0     198  10604  1984-03  0.030556  1.158230e+07          0           0   
1     189  11607  1984-03  0.045685  1.454463e+06          0           0   
2     169  11703  1984-03  0.039578  1.176425e+07          0           0   
3     148  11754  1984-03 -0.062615  1.054266e+07          0           0   
4     157  11850  1984-03  0.003247  3.268050e+07          0           0   
..    ...    ...      ...       ...           ...        ...         ...   
63    181  64346  1984-03  0.052995  5.541696e+05          0           0   
64    139  65569  1984-03  0.023333  3.745400e+05          0           0   
65    138  74617  1984-03 -0.106618  1.222351e+06          0           0   
66    150  80814  

In [128]:
'''
Helper functions
'''

# Given a group of stocks, calculate equal-weighted and value-weighted weights
def calc_weights(group):
    if group['rebalance'].sum() > 0:
        # Calc equal weights
        group['weights_eq'] = 1 / float(group['PERMNO'].count())
        assert(group['PERMNO'].count() == group['PERMNO'].nunique())
        # Calc value weights
        group['weights_val'] = group['MV'] / group['MV'].sum()
        return group
    else:
        group['weights_eq'] = np.nan
        group['weights_val'] = np.nan
        return group

# Calculates returns and prints the returns mean, vol, and Sharpe ratio for a strategy
def analyze(returns, strat_name):
    strat_mean = returns.mean()
    strat_vol = returns.std()
    strat_sharpe = strat_mean / strat_vol
    print(f"{strat_name} monthly returns:\nMean = {strat_mean}%\nVolatility = {strat_vol}%\nSharpe Ratio = {strat_sharpe}")
    
# Estimates the CAPM and FF3 models on df_old using the returns found in ret_col_name
def estimate_models(df_old, return_col_name, ff5):
    df = pd.merge(df_old, ff5, how='inner', on=['date'])
    assert(len(df) == len(df_old))
    
    # Estimate CAPM
    print('CAPM')
    print(sm.OLS(df[return_col_name] - df['RF'], sm.add_constant(df[['Mkt-RF']])).fit().summary())
    
    # Estimate FF3
    print('FF3')
    print(sm.OLS(df[return_col_name] - df['RF'], sm.add_constant(df[['Mkt-RF', 'SMB', 'HML']])).fit().summary())
    
    # Estimate Carhart
    print('Carhart')
    # print(sm.OLS(df[return_col_name] - df['RF'], sm.add_constant(df[['Mkt-RF', 'SMB', 'HML', 'nuggets']])).fit().summary())
    
    # Estimate FF5
    print('FF5')
    print(sm.OLS(df[return_col_name] - df['RF'], sm.add_constant(df[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']])).fit().summary())
    
# Plots the cumulative returns for a strategy versus the CAPM-implied returns
def plot_cum_returns(df, return_col_name, capm_beta):
    df[return_col_name + '_MIR'] = df['RF'] + capm_beta * df['Mkt-RF']
    
    dates = df_total['date'] // 100 + (df_total['date'] % 100) / 12
    strategy_cumulative = (df[return_col_name] / 100 + 1.0).cumprod()
    mir_cumulative = (df[return_col_name + '_MIR'] / 100 + 1.0).cumprod()

    plt.figure()
    plt.plot(dates, strategy_cumulative, label=f'{return_col_name} Portfolio Value')
    plt.plot(dates, mir_cumulative, label=f'CAPM-Implied Portfolio Value')

    plt.title(f'{return_col_name} Model Performance')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Portfolio Value')

    plt.legend()
    plt.show()

In [132]:
'''
Load FF5 data
'''

ff5 = pd.read_csv('ff5_factors.csv')
ff5 = ff5.astype({'date': 'string'})
ff5['date'] = ff5['date'].apply(lambda x: x[:4] + '-' + x[4:])

In [142]:
ret_df['RET'] *= 100
ret_df['weighted_val_ret'] = ret_df['weights_val'] * ret_df['RET']
ret_df['weighted_eq_ret'] = ret_df['weights_eq'] * ret_df['RET']
ret_df = ret_df.sort_values(['date', 'PERMNO'])

eq_returns = ret_df.groupby('date')['weighted_eq_ret'].sum()
val_returns = ret_df.groupby('date')['weighted_val_ret'].sum()

analyze(eq_returns, "Equal-weighted Best Companies")
print()
analyze(val_returns, "Value-weighted Best Companies")

Equal-weighted Best Companies monthly returns:
Mean = 1.2420531431250874%
Volatility = 5.470492078659585%
Sharpe Ratio = 0.22704596319046738

Value-weighted Best Companies monthly returns:
Mean = 1.2035760193169183%
Volatility = 5.268359495233678%
Sharpe Ratio = 0.22845366198069852


In [145]:
eq_capm_beta = estimate_models(eq_returns, 'weighted_eq_ret', ff5)

CAPM
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.891
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                     3622.
Date:                Wed, 24 Apr 2024   Prob (F-statistic):          9.11e-215
Time:                        21:19:41   Log-Likelihood:                -890.01
No. Observations:                 443   AIC:                             1784.
Df Residuals:                     441   BIC:                             1792.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1181      0.087      1.356    

In [146]:
val_capm_beta = estimate_models(val_returns, 'weighted_val_ret', ff5)


CAPM
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.831
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                     2176.
Date:                Wed, 24 Apr 2024   Prob (F-statistic):          1.22e-172
Time:                        21:20:02   Log-Likelihood:                -969.87
No. Observations:                 443   AIC:                             1944.
Df Residuals:                     441   BIC:                             1952.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1403      0.104      1.346    